In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pickle
from functions import get_dyn_v2, get_prop_abs_v2, create_R_v1, get_var_hist,simulate_traj,get_prop_abs, create_R,get_theo_var_l,get_x2_var
from functions import get_non_delayed_prop, get_non_delayed_dyn
from setup import setup_matplotlib
#from scipy.linalg import expm
#from scipy.sparse import csr_array, coo_array
from scipy.optimize import curve_fit

setup_matplotlib()

np.random.seed(42)

In [3]:
k = 1
border = 1
N_x_border = 6.5
max_t = 10
ntau = 3

dx = border/N_x_border
print('dx',dx)
sb = 17.5*dx
sb -= sb%dx
print(sb)

f = 30
N_p = 2_000
N_loop = 10
eq_perc = 0.75
eq_f = 0.75


x_s = np.arange(-sb,border-dx/4,dx) #don't include border !
bins = np.arange(-sb-dx/2,border+dx/4,dx)
N_x = len(x_s)
print(N_x, len(bins))



def force(x,*args):
     return -k*x

i_zero = np.argmin((x_s)**2)

s_s = np.arange(0.2,0.76,0.05)
ltaus = np.concatenate(([0,.05], np.arange(0.2,2.2,0.2) ))
#ltaus = np.array([0])

dx 0.15384615384615385
2.6153846153846154
24 25


In [4]:
x_s, bins

(array([-2.61538462e+00, -2.46153846e+00, -2.30769231e+00, -2.15384615e+00,
        -2.00000000e+00, -1.84615385e+00, -1.69230769e+00, -1.53846154e+00,
        -1.38461538e+00, -1.23076923e+00, -1.07692308e+00, -9.23076923e-01,
        -7.69230769e-01, -6.15384615e-01, -4.61538462e-01, -3.07692308e-01,
        -1.53846154e-01, -1.77635684e-15,  1.53846154e-01,  3.07692308e-01,
         4.61538462e-01,  6.15384615e-01,  7.69230769e-01,  9.23076923e-01]),
 array([-2.69230769, -2.53846154, -2.38461538, -2.23076923, -2.07692308,
        -1.92307692, -1.76923077, -1.61538462, -1.46153846, -1.30769231,
        -1.15384615, -1.        , -0.84615385, -0.69230769, -0.53846154,
        -0.38461538, -0.23076923, -0.07692308,  0.07692308,  0.23076923,
         0.38461538,  0.53846154,  0.69230769,  0.84615385,  1.        ]))

In [5]:
test_ts = np.linspace(0,max_t, 4000)

for j,s in enumerate(tqdm(s_s)):
    D = s**2/2
    exact_eqtimes = [test_ts[np.argmin((get_theo_var_l(test_ts,ltau,D) - eq_perc*get_x2_var(ltau, k, s))**2)] for ltau in ltaus]

    num_rates = np.empty((len(ltaus),2))
    sim_rates = np.empty((len(ltaus),2))

    for i,ltau in enumerate(tqdm(ltaus, leave = False)): 
        ldt = ltau/ntau if ltau > 0 else 0.01
        q_f = np.log(1-eq_f)/np.log(1-eq_perc)
        rel_time = q_f*exact_eqtimes[i]



        N_t = max(int(2*rel_time/ldt),int(max_t/ldt), 30)
        ts = np.arange(0,N_t)*ldt
        sim_ts = np.arange(0,f*N_t+ntau*f)*ldt/f-ltau

        pos = simulate_traj(N_p=N_p, N_loop=N_loop, N_t=(N_t+ntau)*f, ntau=ntau*f, s=s, dt=ldt/f,border=0, force=force)
        pos_abs = pos.copy()
        for l in range(ntau+1, pos.shape[-1]):
            pos_abs[(pos_abs[:,:,l] > border),l:] = np.nan

        sim_hists = np.swapaxes(np.apply_along_axis(lambda a: np.histogram(a, bins)[0], 1, pos_abs), 1,2)

        if ltau > 0:
            prop = get_prop_abs_v2(x_s, force,D,ldt,dx)
            R, _, end_states = create_R_v1(N_x, ntau, prop)

            _, hists = get_dyn_v2(R, i_zero,N_t, N_x, ntau, end_states)
        else:
            R = get_non_delayed_prop(x_s, force,D,ldt,dx)
            hists = get_non_delayed_dyn(R,i_zero,N_t, N_x)


        num_sur = np.sum(hists, axis = 1)
        sim_sur = np.sum(sim_hists, axis = 2)/N_p

        sim_cond = (sim_ts > rel_time) * (sim_sur > 0.05)
        num_cond = (ts > rel_time) * (num_sur > 0.05)

        sim_rate = [-np.mean(np.diff(np.log(sim_sur[i,this_sim_cond]))/ldt*f) for i,this_sim_cond in enumerate(sim_cond)]
        num_rate = -np.mean(np.diff(np.log(num_sur[num_cond]))/ldt)

        sim_rates[i] = (np.mean(sim_rate), np.std(sim_rate)/np.sqrt(N_loop))
        num_rates[i] = (num_rate, 0)
        
    with open(f'nummeric_results/cusp_abs_s{s}.pkl', 'wb') as file:
        pickle.dump({'ltaus':ltaus,
                     'num_rates':num_rates,
                    'sim_rates':sim_rates
                    }, file)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
plt.errorbar(ltaus,*sim_rates.T)
plt.errorbar(ltaus,*num_rates.T)

In [ ]:
plt.plot(sim_ts, sim_sur[0], alpha =.5)
plt.plot(ts, num_sur, '--', alpha =.5)
plt.plot(sim_ts[sim_cond[0]], sim_sur[0][sim_cond[0]])
plt.plot(ts[num_cond], num_sur[num_cond], '--')

In [ ]:
idx = len(hists)-1
t_hist = sim_hists.mean(0)
plt.plot(x_s,t_hist[(idx+ntau)*f]/N_p)
plt.plot(x_s,hists[idx], '--')